In [1]:
from scipy.optimize import linprog
import numpy as np

In [2]:
# Функция для вычисления столбца, в котором находится наименьшее значение
def find_pivot_column(tableau):
    return tableau[-1].index(min(tableau[-1]))


# Функция для вычисления строки, в которой находится минимальное отношение b[i]/A[i][pivot_column]
def find_pivot_row(tableau, pivot_column):
    ratios = [tableau[i][-1] / tableau[i][pivot_column] if tableau[i][pivot_column] > 0 else float('inf') for i in
              range(len(tableau) - 1)]
    return ratios.index(min(ratios))


# Функция для обновления таблицы симплекс-метода
def update_tableau(tableau, pivot_row, pivot_column):
    pivot_element = tableau[pivot_row][pivot_column]
    new_tableau = []

    for i in range(len(tableau)):
        if i == pivot_row:
            new_row = [x / pivot_element for x in tableau[i]]
        else:
            factor = tableau[i][pivot_column] / pivot_element
            new_row = [tableau[i][j] - factor * tableau[pivot_row][j] for j in range(len(tableau[i]))]
        new_tableau.append(new_row)

    return new_tableau


# Функция для проверки, завершен ли алгоритм
def is_optimal(tableau):
    return all(x >= 0 for x in tableau[-1])


def print_tableau(tableau):
    for row in tableau:
        print(row)
    print()


# Функция для нахождения оптимального решения
def simplex_method(A, b, c):
    # Создаем начальную таблицу симплекс-метода
    tableau = [row[:] + [x] for row, x in zip(A, b)]
    tableau.append(c[:] + [0])

    # Выводим начальную таблицу
    print("Начальная таблица:")
    print_tableau(tableau)

    # Продолжаем итерации симплекс-метода, пока не будет достигнуто оптимальное значение
    iteration = 1
    while not is_optimal(tableau):
        pivot_column = find_pivot_column(tableau)
        pivot_row = find_pivot_row(tableau, pivot_column)
        tableau = update_tableau(tableau, pivot_row, pivot_column)
        print(f"Итерация {iteration}:")
        print_tableau(tableau)
        iteration += 1

    return tableau


In [3]:
# Коэффициенты целевой функции
c = [-1.1, -1.5]  # Мы максимизируем прибыль, поэтому коэффициенты отрицательные

# Коэффициенты левых частей ограничений
A = [
    [1, 2],    # Ограничение на время работы
    [1, 0],    # Ограничение по производству деталей типа А
    [0, 2],    # Ограничение по производству деталей типа В
    [2, 4],    # Ограничение по использованию полимерного материала
    [5, 3],    # Ограничение по использованию листового материала
    [-1, -1]   # Минимальное количество производимых деталей
]

# Правые части ограничений
b = [5000, 2500, 2000, 10000, 10000, -1500]

# Задаем границы для переменных x1 и x2 (неотрицательные значения)
x0_bounds = (0, None)
x1_bounds = (0, None)

In [4]:
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds], method='highs')

print("Максимальный доход:", -res.fun, "руб.")
print("Количество деталей типа А:", round(res.x[0]), "шт")
print("Количество деталей типа В:", round(res.x[1]), "шт")


Максимальный доход: 3040.0 руб.
Количество деталей типа А: 1400 шт
Количество деталей типа В: 1000 шт


In [5]:
tableau = simplex_method(A, b, c)

for sublist in tableau:
    if sublist[0] == 1.0:
        A_ans = (sublist[2])

for sublist in tableau:
    if sublist[1] == 1.0:
        B_ans = (sublist[2])
    
max_val = tableau[-1][-1]

if is_optimal(tableau):
    print("Максимальный доход:", round(max_val), "руб.")
    print("Количество деталей типа А:", round(A_ans), "шт")
    print("Количество деталей типа В:", round(B_ans), "шт")
else:
    print("Решение не найдено.")

Начальная таблица:
[1, 2, 5000]
[1, 0, 2500]
[0, 2, 2000]
[2, 4, 10000]
[5, 3, 10000]
[-1, -1, -1500]
[-1.1, -1.5, 0]

Итерация 1:
[1.0, 0.0, 3000.0]
[1.0, 0.0, 2500.0]
[0.0, 1.0, 1000.0]
[2.0, 0.0, 6000.0]
[5.0, 0.0, 7000.0]
[-1.0, 0.0, -500.0]
[-1.1, 0.0, 1500.0]

Итерация 2:
[0.0, 0.0, 1600.0]
[0.0, 0.0, 1100.0]
[0.0, 1.0, 1000.0]
[0.0, 0.0, 3200.0]
[1.0, 0.0, 1400.0]
[0.0, 0.0, 900.0]
[0.0, 0.0, 3040.0]

Максимальный доход: 3040 руб.
Количество деталей типа А: 1400 шт
Количество деталей типа В: 1000 шт


In [6]:
if int(max_val) == int(-res.fun) and int(res.x[0]) == int(A_ans) and int(res.x[1]) == int(B_ans):
    print("Результаты совпали")
else:
    print("Результаты не совпали")

Результаты совпали
